# Interactive Plotting

In [ ]:
import pandas as pd
import datetime as dt
import math
from ast import literal_eval
import bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual
from bokeh.io import output_notebook
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

## Data

Chicago data portal

List of 'L' Stops:
https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops-Map/zbnc-zirh
Station entry averages:
https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Monthly-Day-Type-A/t2rn-p8d7

In [ ]:
def get_data_chicago(id):
    '''
    Connect to the chicago data portal API and returns a dataframe
    '''
    
    url = f'https://data.cityofchicago.org/api/views/{id}/rows.csv?accessType=DOWNLOAD'
    df = pd.read_csv(url)
    
    return df

In [ ]:
#Ridership data
rides = get_data_chicago('t2rn-p8d7')
rides['month_beginning'] = pd.to_datetime(rides['month_beginning'])
print(rides.shape)
rides.head()

In [ ]:
#CTA Transit Station Location Data
stations = get_data_chicago('zbnc-zirh')
print(stations.shape)
stations.head()

In [ ]:
stations.drop_duplicates(subset='MAP_ID', keep="last", inplace=True)

## Bokeh building blocks

A Bokeh plot as a series of objects. At the very base layer is a Figure, and on top of it you can add Glyphs.

* Figure: grouping of all the elements (i.e. the plot)
* Glyphs: basic visual markers that Bokeh can display

## Plot ridership by month

In [ ]:
rides_by_month = rides[['month_beginning', 'monthtotal']].groupby('month_beginning').mean()
rides_by_month.head()

In [ ]:
plot = figure(title='Average rides per month', x_axis_label='Date', 
              y_axis_label='Month average', x_axis_type='datetime', plot_height=400)
plot.line(rides_by_month.index, rides_by_month['monthtotal'], line_width=4, color='indianred')
show(plot)

## Plot popular stations

In [ ]:
rides_by_station = rides[['stationame', 'monthtotal']].groupby('stationame').mean()
rides_by_station = rides_by_station.sort_values(by='monthtotal', ascending=False).head()
rides_by_station

In [ ]:
p = figure(x_range=list(rides_by_station.index), title="Top Stations", plot_height=250)
p.vbar(x=rides_by_station.index, top=rides_by_station['monthtotal'], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [ ]:
# add range slider

def plot_stations(df, num_bars):
    rides_by_station = df[['stationame', 'monthtotal']].groupby('stationame').mean()
    rides_by_station = rides_by_station.sort_values(by='monthtotal', ascending=False).head(num_bars)
    
    plot = figure(x_range=list(rides_by_station.index), title="Top Stations", plot_height=350)
    plot.vbar(x=rides_by_station.index, top=rides_by_station['monthtotal'], width=0.9)
    
    plot.xaxis.major_label_orientation = "vertical"
    plot.xgrid.grid_line_color = None
    plot.y_range.start = 0
    
    return plot

@interact(value=(0, 40))
def make_plot_stations(value=3):
    plot = plot_stations(rides, value)
    show(plot)

## Interactive Maps

What part of the city has the most train ridership?

In [ ]:
stations.head()

Adding map data to a plot uses the same methodology as adding other types of Glyphs to a Figure. This time  you will be passing in tiles using the add_tiles() command, along with a tile provider as an argument.

In [ ]:
tile_provider = get_provider(CARTODBPOSITRON)

p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
show(p)

<b>Mercator Projection:</b> The tiles for the map uses the mercator projection. The the spherical coordinates of the earth (latitude and longitude) are projected onto a plane (X and Y coordinates). https://en.wikipedia.org/wiki/Mercator_projection

In [ ]:
def merc(Coords):
    '''
    Takes a string of latitude and longitude coordinates (like the ones in the dataframe), 
    and converts them to a tuple of X and Y coordinates.
    '''
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
stations['coords_x'] = stations['Location'].apply(lambda x: merc(x)[0])
stations['coords_y'] = stations['Location'].apply(lambda x: merc(x)[1])
stations.head()

We now can overlay the stations on the map we created earlier.

In [ ]:
tile_provider = get_provider(CARTODBPOSITRON)

p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(tile_provider)

p.circle(x = stations['coords_x'],
         y = stations['coords_y'])
show(p)

In [ ]:
rides = rides[['station_id', 'monthtotal']].groupby('station_id').mean()
print(rides.shape)
rides.head()

In [ ]:
merged = stations.merge(rides, how='inner', left_on='MAP_ID', right_index=True)
print(merged.shape)
merged.head()

In [ ]:
#scale the ridership data to the appropriate size circle
merged['circle_sizes'] = merged['monthtotal'] / 10000

In [ ]:
tile_provider = get_provider(CARTODBPOSITRON)

p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(tile_provider)

p.circle(x=merged['coords_x'],
         y=merged['coords_y'], 
         size=merged['circle_sizes'],
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05)

show(p)

Bokeh has interactive tools that can be used to report information. We can use the <b>Hover Tool</b> to overlay information about each station when a user hovers over a circle.

In [ ]:
def map_ridership(df):
    
    tile_provider = get_provider(CARTODBPOSITRON)


    source = ColumnDataSource(data=dict(
                            x=list(df['coords_x']), 
                            y=list(df['coords_y']),
                            ridership=list(merged['monthtotal']),
                            sizes=list(df['circle_sizes']),
                            stationname=list(df['STATION_NAME'])))

    hover = HoverTool(tooltips=[
        ("station", "@stationname"),
        ("ridership","@ridership")
    
    ])

    p = figure(x_axis_type="mercator", 
               y_axis_type="mercator",
               tools=[hover, 'wheel_zoom','save'])

    p.add_tile(get_provider(tile_provider))

    p.circle(x='x',
             y='y', 
             source=source,
             size='sizes',
             line_color="#FF0000", 
             fill_color="#FF0000",
             fill_alpha=0.05)
    return p
    
plot = map_ridership(merged)
show(plot)